# Data 1 | Clean Voterfiles

This does two things with data retrieved from https://dl.ncsbe.gov/?prefix=data/.

First since the turnout history file is subject to periodic purges, I merge all the history files available. 

Second, I clean and chunk the snapshots. To do this I unzip the text files, open the text files as dataframe, the delete the textfile. Then I clean the dataframe:
1. Generate new variables
2. Rename necessary variables
3. Drop unnecessary variables
4. Reformat the necessary variables
5. Then generate new variables using the reformatted variables


## Merge Turnout Files

In [1]:
from datalink import *

hists = [x for x in os.listdir(path_0 + 'history') if 'ncvhis_Statewide' in x and '.txt' in x]
hists = [pd.read_csv(path_0 + 'history/'+x, sep='\t', header=0) for x in hists]
print('Loaded History Files')

hist = pd.concat(hists, ignore_index=True)
del hists
print('Concat Finished')

hist = hist.drop_duplicates()
print('Dropped Duplicates')

hist.to_pickle(path_0 + 'history/turnout_history.pkl')

### Check That Chunks Replicate

In [5]:
c = pd.read_pickle('data1_clean/NC/20201103_chunk_2.pkl')

In [10]:
co = pd.read_pickle('data1_clean/NC/year_date/2020_chunk_2.pkl')

In [11]:
co.idu.unique() == c.idu.unique()

array([ True,  True,  True, ...,  True,  True,  True])

## Create Clean Chunks

next, run datalink again to use these new chunks!!!







In [1]:
from data1_clean import *

history = pd.read_pickle(f'{path_0}/history/turnout_history.pkl')
history['idu'] = history['ncid'].astype(str) + history['voter_reg_num'].astype(str)

for date in dates[-2:]:
    print(date)
    file, year = zip_map[date], year_map[date]
    
    if f'{date}_chunk_0.pkl' in os.listdir(path_1):
        print('  Done')
        
    if int(year) < 2010:
        print('  Too Old (for now)')
        
    if (f'{date}_chunk_0.pkl' not in os.listdir(path_1)) and (int(year) >= 2010):
        
        """ Unzip Snapshot """
        
        os.system(f'unzip -o {path_0_zip}{file} -d {path_0_zip}')
        zipfile = ZipFile(f'{path_0_zip}{file}')
        extracted_file, = zipfile.namelist()

        """ Chunk and Clean Snapshot """

        voter_chunks = pd.read_csv(path_0_zip + extracted_file, 
                                    low_memory=False, 
                                    iterator=True, 
                                    chunksize=10**6, 
                                    sep='\t', 
                                    encoding='utf-16-le', 
                                    encoding_errors='ignore', 
                                    error_bad_lines=False, 
                                    quoting=csv.QUOTE_NONE)
        os.remove(path_0_zip + extracted_file)
        
        """ tail -n 100 ./file.txt shows the last 100 lines in terminal"""
        
        """
        try:
            voter_chunks = pd.read_csv(path_0_zip + extracted_file, 
                                       low_memory=False, 
                                       iterator=True, 
                                       chunksize=10**6, 
                                       sep='\t', 
                                       encoding='utf-16', 
                                       encoding_errors='ignore', 
                                       error_bad_lines=False, 
                                       quoting=csv.QUOTE_NONE)
        except:
        print('Not encoding utf-16')
        """
        
        for index, voters in enumerate(voter_chunks):
            print(f'  Cleaning Chunk {index}')
            voters = clean_NC(voters, history, date, nc_rename, run_elections=True)
            print(f'  Saving Chunk {index}')

            voters.to_pickle(f'{path_1}{date}_chunk_{index}.pkl')

20191105
Archive:  data0_raw/NC/snapshots/VR_Snapshot_20191105.zip
  inflating: data0_raw/NC/snapshots/VR_Snapshot_20191105.txt  


/home/taylor/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


  Cleaning Chunk 0
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate


/media/taylor/Main/Dropbox/voter_data/data1_clean.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  voters['voted_' + election] = voters.idu.isin(election_data.idu.unique())*1


  Saving Chunk 0
  Cleaning Chunk 1
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 1
  Cleaning Chunk 2
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 2
  Cleaning Chunk 3
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 3
  Cleaning Chunk 4
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 4
  Cleaning Chunk 5
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 5
  Cleaning Chunk 6
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 6
  Cleaning Chunk 7
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 7
  Cleaning Chunk 8
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 8
  Cleaning Chunk 9
  || Generate
  || Drop
  || Rename
  || Reformat
  || Post Generate
  Saving Chunk 9
  Cleaning Chunk 10
  || Generate
  ||

In [ ]:
        """
        try:
            voter_chunks = pd.read_csv(path_0_zip + extracted_file, 
                                       low_memory=False, 
                                       iterator=True, 
                                       chunksize=10**6, 
                                       sep='\t', 
                                       encoding='utf-16', 
                                       encoding_errors='ignore', 
                                       error_bad_lines=False, 
                                       quoting=csv.QUOTE_NONE)
        except:
        print('Not encoding utf-16')
        """